## Imports, masks, conversions

In [1]:
import xarray as xr
import os
import sys
sys.path.append('/glade/u/home/jzweifel/utils')
import importlib
import load_variable
importlib.reload(load_variable)
from load_variable import load_variables_for_all_years
import pandas as pd

NPAC = xr.open_dataarray('/glade/u/home/jzweifel/jupyter_workspaces/Masters_paper_analysis/weighting_and_masks/NPO_binary_map_lower_bound_20_deg.nc')
East_Asia = xr.open_dataarray('/glade/u/home/jzweifel/jupyter_workspaces/Masters_paper_analysis/weighting_and_masks/CJK_binary_assign_coords.nc')
area = xr.open_dataarray('/glade/u/home/jzweifel/jupyter_workspaces/Masters_paper_analysis/weighting_and_masks/AREA.nc')

second = 1
minute = 60 * second
hour = 60 * minute
day = 24 * hour
seconds_per_year = 365 * day

kg = 1
Tg_per_kg = 1e-9 * kg


## Budget variables

In [2]:
emission_vars = ['SFso4_a1', 'SFso4_a2', 'SFso4_a3', 'so4_a1_CMXF', 'so4_a2_CMXF']
# surface fluxes & vertically integrated external forcing (kg/m2/s)

aq_prod_vars = ['so4_c1AQSO4', 'so4_c1AQH2SO4', 'so4_c2AQSO4', 'so4_c2AQH2SO4', 'so4_c3AQSO4', 'so4_c3AQH2SO4']
# aqueous phase chemistry (kg/m2/s)

dry_chem_nuc_vars = ['so4_a2_sfnnuc1']
# new particle nucleation column tendency (kg/m2/s)

dry_chem_cond_vars = ['so4_a1_sfgaex1', 'so4_a2_sfgaex1', 'so4_a3_sfgaex1']
# gas-aerosol-exchange primary column tendency (kg/m2/s)

dry_dep_variables = ['so4_a1DDF', 'so4_a2DDF', 'so4_a3DDF', 'so4_c1DDF', 'so4_c2DDF', 'so4_c3DDF']
# dry deposition flux at bottom (grav + turb) (kg/m2/s)

wet_dep_variables = ['so4_a1SFWET', 'so4_a2SFWET', 'so4_a3SFWET', 'so4_c1SFWET', 'so4_c2SFWET', 'so4_c3SFWET']
# wet deposition flux at surface (kg/m2/s)

## Function to compute regional budget change across time between simulations

In [3]:
def compute_budget_change(data_dict, region_mask, year1, year0, area, seconds_per_year, Tg_per_kg, flip_sign=False):
    
    # difference
    diff = data_dict[year1].where(region_mask) - data_dict[year0].where(region_mask)
    
    # sign convention
    if flip_sign:
        diff = diff * -1
    
    # scale and reduce
    result = (diff * area * seconds_per_year * Tg_per_kg).mean(dim=['time']).sum(dim=['lat', 'lon']).to_array().sum()
    
    return result

## Computing Budgets:
### NPAC and East Asia (2006 simulation - 1970 simulation)

In [6]:
import pandas as pd

data_groups = {
    "emission": load_variables_for_all_years('h1', emission_vars),
    "aq_prod": load_variables_for_all_years('h1', aq_prod_vars),
    "dry_chem_nuc": load_variables_for_all_years('h1', dry_chem_nuc_vars),
    "dry_chem_cond": load_variables_for_all_years('h1', dry_chem_cond_vars),
    "dry_dep": load_variables_for_all_years('h1', dry_dep_variables),
    "wet_dep": load_variables_for_all_years('h1', wet_dep_variables),
}

regions = {
    "NPAC": NPAC,
    "East_Asia": East_Asia,
}

years = ("2006", "1970")

budgets = {}

for category, dataset in data_groups.items():
    budgets[category] = {}
    for region_name, mask in regions.items():
        flip = (category == "wet_dep")  # flip sign only for wet dep
        budgets[category][region_name] = compute_budget_change(dataset, mask, *years, area, seconds_per_year, Tg_per_kg, flip_sign=flip).item()

df = pd.DataFrame(budgets).T

In [7]:
df

,NPAC,East_Asia
emission,-0.000011,1.278880
aq_prod,1.190142,5.405908
dry_chem_nuc,0.000755,0.003819
dry_chem_cond,0.357251,7.139449
dry_dep,0.231437,1.752965
wet_dep,2.451432,8.400840


In [85]:
a = load_variables_for_all_years('h1', emission_vars)
b = load_variables_for_all_years('h1', aq_prod_vars)
c = load_variables_for_all_years('h1', dry_chem_nuc_vars)
d = load_variables_for_all_years('h1', dry_chem_cond_vars)
e = load_variables_for_all_years('h1', dry_dep_variables)
f = load_variables_for_all_years('h1', wet_dep_variables)

NPAC_emission_budget = ((a['2006'].where(NPAC) - a['1970'].where(NPAC))*area*seconds_per_year*Tg_per_kg).mean(dim=['time']).sum(dim=['lat', 'lon']).to_array().sum()
East_Asia_emission_budget = ((a['2006'].where(East_Asia) - a['1970'].where(East_Asia))*area*seconds_per_year*Tg_per_kg).mean(dim=['time']).sum(dim=['lat', 'lon']).to_array().sum()

NPAC_aq_prod_budget = ((b['2006'].where(NPAC) - b['1970'].where(NPAC))*area*seconds_per_year*Tg_per_kg).mean(dim=['time']).sum(dim=['lat', 'lon']).to_array().sum()
East_Asia_aq_prod_budget = ((b['2006'].where(East_Asia) - b['1970'].where(East_Asia))*area*seconds_per_year*Tg_per_kg).mean(dim=['time']).sum(dim=['lat', 'lon']).to_array().sum()

NPAC_dry_chem_nuc_budget = ((c['2006'].where(NPAC) -c['1970'].where(NPAC))*area*seconds_per_year*Tg_per_kg).mean(dim=['time']).sum(dim=['lat', 'lon']).to_array().sum()
East_Asia_dry_chem_nuc_budget = ((c['2006'].where(East_Asia) - c['1970'].where(East_Asia))*area*seconds_per_year*Tg_per_kg).mean(dim=['time']).sum(dim=['lat', 'lon']).to_array().sum()

NPAC_dry_chem_cond_budget = ((d['2006'].where(NPAC) - d['1970'].where(NPAC))*area*seconds_per_year*Tg_per_kg).mean(dim=['time']).sum(dim=['lat', 'lon']).to_array().sum()
East_Asia_dry_chem_cond_budget = ((d['2006'].where(East_Asia) - d['1970'].where(East_Asia))*area*seconds_per_year*Tg_per_kg).mean(dim=['time']).sum(dim=['lat', 'lon']).to_array().sum()

NPAC_dry_dep_budget = ((e['2006'].where(NPAC) - e['1970'].where(NPAC))*area*seconds_per_year*Tg_per_kg).mean(dim=['time']).sum(dim=['lat', 'lon']).to_array().sum()
East_Asia_dry_dep_budget = ((e['2006'].where(East_Asia) - e['1970'].where(East_Asia))*area*seconds_per_year*Tg_per_kg).mean(dim=['time']).sum(dim=['lat', 'lon']).to_array().sum()

NPAC_wet_dep_budget = ((f['2006'].where(NPAC)*-1 - f['1970'].where(NPAC)*-1)*area*seconds_per_year*Tg_per_kg).mean(dim=['time']).sum(dim=['lat', 'lon']).to_array().sum()
East_Asia_wet_dep_budget = ((f['2006'].where(East_Asia)*-1 - f['1970'].where(East_Asia)*-1)*area*seconds_per_year*Tg_per_kg).mean(dim=['time']).sum(dim=['lat', 'lon']).to_array().sum()

